In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

['test.csv', 'test_images', 'train.csv', 'sample_submission.csv', 'train_images']


In [2]:
train_df = pd.read_csv('../input/train.csv')
print(train_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3662 entries, 0 to 3661
Data columns (total 2 columns):
id_code      3662 non-null object
diagnosis    3662 non-null int64
dtypes: int64(1), object(1)
memory usage: 57.3+ KB
None


In [3]:
train_df['diagnosis'] = train_df['diagnosis'].astype('str')
train_df['id_code'] = train_df['id_code'].astype(str)+'.png'
from keras.preprocessing.image import ImageDataGenerator

datagen=ImageDataGenerator(
    rescale=1./255, 
    validation_split=0.2)

batch_size = 32

train_gen=datagen.flow_from_dataframe(
    dataframe=train_df,
    directory="../input/train_images",
    x_col="id_code",
    y_col="diagnosis",
    batch_size=batch_size,
    shuffle=True,
    class_mode="categorical",
    target_size=(160,160),
    subset='training')

test_gen=datagen.flow_from_dataframe(
    dataframe=train_df,
    directory="../input/train_images",
    x_col="id_code",
    y_col="diagnosis",
    batch_size=batch_size,
    shuffle=True,
    class_mode="categorical", 
    target_size=(160,160),
    subset='validation')

Using TensorFlow backend.


Found 2930 validated image filenames belonging to 5 classes.
Found 732 validated image filenames belonging to 5 classes.


In [4]:
y_train = train_df['diagnosis']
from keras.utils import np_utils
y_train = np_utils.to_categorical(y_train)
num_classes = y_train.shape[1]

In [5]:
from keras.models import Sequential
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation
from keras.layers.core import Flatten
from keras.layers.core import Dropout
from keras.layers.core import Dense
from keras import backend as K

In [6]:
model = Sequential()
model.add(Conv2D(64, (3, 3), padding="same",
			input_shape=(160,160,3)))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=-1))
model.add(Conv2D(64, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=-1))

model.add(MaxPooling2D(pool_size=(3, 3),strides=2))
model.add(Dropout(0.25))
model.add(Conv2D(128, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=-1))
model.add(Conv2D(128, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=-1))
model.add(MaxPooling2D(pool_size=(3, 3),strides=2))
model.add(Dropout(0.25))
model.add(Conv2D(256, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=-1))
model.add(Conv2D(256, (3, 3), padding="valid"))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=-1))

model.add(Conv2D(256, (3, 3), padding="valid"))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=-1))
model.add(Conv2D(256, (3, 3), padding="valid"))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=-1))
model.add(MaxPooling2D(pool_size=(3, 3),strides=2))
model.add(Dropout(0.25))
model.add(Conv2D(512, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=-1))
model.add(Conv2D(512, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=-1))
model.add(Conv2D(512, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=-1))
model.add(Conv2D(512, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=-1))
model.add(MaxPooling2D(pool_size=(3,3),strides=2))
model.add(Dropout(0.25))

model.add(Conv2D(512, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=-1))
model.add(Conv2D(512, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=-1))
model.add(Conv2D(512, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=-1))
model.add(Conv2D(512, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=-1))
model.add(MaxPooling2D(pool_size=(3,3),strides=2))
model.add(Dropout(0.25))




In [7]:
model.add(Flatten())
model.add(Dense(1024))
model.add(Activation("relu"))
model.add(BatchNormalization())
model.add(Dense(num_classes, activation='softmax'))
model.compile(loss='categorical_crossentropy',optimizer='Adam',metrics=['accuracy'])

In [8]:
from keras.callbacks import EarlyStopping, ModelCheckpoint
es= EarlyStopping(monitor='val_loss', mode ='min', verbose = 0, patience = 20)
mc = ModelCheckpoint('model.h5', monitor='val_loss', save_best_only = True, mode ='min', verbose = 0)

In [9]:
model.fit_generator(generator=train_gen,              
                                    steps_per_epoch=len(train_gen),
                                    validation_data=test_gen,                    
                                    validation_steps=len(test_gen),
                                    epochs=10,
                                    callbacks = [es, mc], 
                                    use_multiprocessing = True,
                                    verbose=1)

Epoch 1/10
92/92 [==============================] - 424s 5s/step - loss: 1.4915 - acc: 0.5300 - val_loss: 2.7040 - val_acc: 0.4057
Epoch 2/10
92/92 [==============================] - 372s 4s/step - loss: 1.0496 - acc: 0.6286 - val_loss: 1.0777 - val_acc: 0.6366
Epoch 3/10
92/92 [==============================] - 371s 4s/step - loss: 0.9246 - acc: 0.6662 - val_loss: 1.0171 - val_acc: 0.6175
Epoch 4/10
92/92 [==============================] - 373s 4s/step - loss: 0.9213 - acc: 0.6791 - val_loss: 0.9333 - val_acc: 0.6735
Epoch 5/10
92/92 [==============================] - 368s 4s/step - loss: 0.8915 - acc: 0.6878 - val_loss: 0.8725 - val_acc: 0.6776
Epoch 6/10
92/92 [==============================] - 372s 4s/step - loss: 0.8191 - acc: 0.7067 - val_loss: 0.8744 - val_acc: 0.6339
Epoch 7/10
92/92 [==============================] - 370s 4s/step - loss: 0.8171 - acc: 0.7026 - val_loss: 0.8740 - val_acc: 0.6653
Epoch 8/10
92/92 [==============================] - 368s 4s/step - loss: 0.8251 - a

In [10]:
submission_df = pd.read_csv('../input/sample_submission.csv')
#submission_df['diagnosis'] = submission_df['diagnosis'].astype('str')
submission_df['id_code'] = submission_df['id_code'].astype(str)+'.png'


In [11]:
submission_datagen=ImageDataGenerator(rescale=1./255)
submission_gen=submission_datagen.flow_from_dataframe(
    dataframe=submission_df,
    directory="../input/test_images",
    x_col="id_code",    
    batch_size=batch_size,
    shuffle=False,
    class_mode=None, 
    target_size=(160,160)
)

Found 1928 validated image filenames.


In [12]:
predictions=model.predict_generator(submission_gen, steps = len(submission_gen))

In [13]:
max_probability = np.argmax(predictions,axis=1) 

In [14]:
submission_df['diagnosis'] = max_probability
submission_df.to_csv('submission.csv', index=False)